## pip install

In [2]:
!uv pip install beautifulsoup4

Using Python 3.12.9 environment at: C:\Users\betashort\Documents\GitHub\KEIBA\.venv
Audited 1 package in 46ms


### import

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

#### read html file

In [3]:
html_file = "../dev/data/newspaper_sample.html"

with open(html_file, "r", encoding="utf-8") as file:
    html = file.read()

In [4]:
soup = BeautifulSoup(html, "html.parser")

### 出馬表の馬リストを取得

In [5]:
# get horse list table
horseList_wrapper = soup.find("div", class_="HorseList_Wrapper")
# get horse list
horse_list = horseList_wrapper.find_all("dl", class_="HorseList")

In [6]:
horse_data = horse_list[0]

In [7]:
col_name = ["枠番", "馬番", "馬名", "性齢", "斤量", "騎手", "調教師", "馬体重", "馬体重増減", "オッズ", "人気"]

### 枠番と馬番を取得

In [8]:
# wakuban
waku = horse_data.find("dt", class_="Waku1").text.strip()
# umaban
umaban = horse_data.find("dt", class_="Waku").text.strip()

### 馬の基本情報を取得

In [12]:
# horse info
#horse_info = horse_data.find("dt", class_="Horse_Info")
# wakuban
waku = horse_data.find("dt", class_="Waku1").text.strip()
waku = int(waku)
# umaban
umaban = horse_data.find("dt", class_="Waku").text.strip()
umaban = int(umaban)
# blood
# father
father = horse_data.find("dt", class_="Horse01").text.strip()
# horse_name
horse_name = horse_data.find("dt", class_="Horse02").text.strip()
# mother
mother = horse_data.find("dt", class_="Horse03").text.strip()
# mother_father
mother_father = horse_data.find("dt", class_="Horse04").text.strip()
# stakeholders
stakeholders = horse_data.find_all("dt", class_="Horse05")
# trainer
trainer = stakeholders[0].text.strip()
# owner
owner = stakeholders[1].text.strip()
# breeder
breeder = stakeholders[2].text.strip()

'1'

In [13]:
# 脚質と連闘情報
tmp = horse_data.find("dt", class_="Horse06")
kyakusitu = tmp.find("span").text.strip()  
rotation = tmp.get_text(strip=True).replace(kyakusitu, "").strip()

In [22]:
# 馬体重・馬体重差・人気・オッズ
weight_tag = horse_data.find("dt", class_="Horse07")
weight = weight_tag.find("span", class_="transition-color").get_text(strip=True).split("(")[0].replace("kg", "").strip()
weight = float(weight)
weight_diff = weight_tag.find("span").find("span").text.strip("()")
weight_diff = float(weight_diff)
odds = weight_tag.find("span", class_="OddsDataTxt").text.strip()
odds = float(odds)
popularity = weight_tag.find("span", id=lambda x: x and x.startswith("ninki")).text.strip()
popularity = int(popularity)

### 騎手&斤量情報を取得

In [32]:
jockey_data = horse_data.find("dd", class_="Jockey")
sex = jockey_data.find("span", class_="Barei").find("span").text.strip()
old = jockey_data.find("span", class_="Barei").text.strip().split(" ")[0].replace(sex, "").strip()
old = int(old)
color = jockey_data.find("span", class_="Barei").text.strip().split(" ")[1].strip()

jockey_compatibility = jockey_data.find_all("span")

if len(jockey_compatibility) == 5:
    jockey_change = "変更なし"
    jockey_name = jockey_compatibility[2].text.strip()
    jockey_result = jockey_compatibility[3].text.strip()
    kinryo = jockey_compatibility[4].text.strip()
    kinryo = float(kinryo)

elif len(jockey_compatibility) == 6:
    jockey_change = jockey_compatibility[2].find("span", class_="Change").text.strip()
    jockey_name = jockey_compatibility[2].text.strip().replace(jockey_change, "").strip()
    jockey_result = jockey_compatibility[4].text.strip()
    kinryo = jockey_compatibility[5].text.strip()
    kinryo = float(kinryo)

### 過去実績を取得

In [50]:
past_direction_list = horse_data.find("dd", class_="Past_Wrapper").find("ul", class_="Past_Direction").find_all("li")

In [51]:
len(past_direction_list)

6

In [119]:
# データ格納用
data_rows = []
row = {}
past_direction_list = horse_data.find("dd", class_="Past_Wrapper").find("ul", class_="Past_Direction").find_all("li")
# 各レース情報を抽出
for idx in range(len(past_direction_list) -1):
    box = past_direction_list[idx].find("div", class_="PastBox")
    

    try:

        if box.find("span", class_="Data26"):
            row[f"{idx+1}走前_日付_場所_レース"] = None
            row[f"{idx+1}走前_クラス"] = None
            row[f"{idx+1}走前_レース名"] = None
            row[f"{idx+1}走前_ハンデ"] = None
            row[f"{idx+1}走前_着順"] = None
            row[f"{idx+1}走前_頭数"] = None
            row[f"{idx+1}走前_枠番"] = None
            row[f"{idx+1}走前_人気"] = None
            
            row[f"{idx+1}走前_馬場"] = None 
            row[f"{idx+1}走前_距離"] = None
            row[f"{idx+1}走前_コース"] = None
            row[f"{idx+1}走前_馬場状態"] = None
            row[f"{idx+1}走前_タイム"] = None

            row[f"{idx+1}走前_騎手"] = None
            row[f"{idx+1}走前_斤量"] = None
            row[f"{idx+1}走前_馬体重"] = None
            row[f"{idx+1}走前_増減"] = None

            row[f"{idx+1}走前_前半"] = None
            row[f"{idx+1}走前_コーナー通過"] = None
            row[f"{idx+1}走前_後半"] = None
            row[f"{idx+1}走前_ペース"] = None
            row[f"{idx+1}走前_馬名"] = None
            row[f"{idx+1}走前_着差"] = None
            row[f"{idx+1}走前_脚質"] = None
        
        else:
            lines = box.find_all("div", class_="PastDataLine")
            row[f"{idx+1}走前_日付_場所_レース"] = lines[0].find("span", class_="Data01").text.strip()
            row[f"{idx+1}走前_クラス"] = lines[0].find("span", class_="Icon_GradeType").text.strip()

            race_name_tag = lines[1].find("span", class_="RaceName").find("a")
            row[f"{idx+1}走前_レース名"] = race_name_tag.text.strip() if race_name_tag else ""
            row[f"{idx+1}走前_ハンデ"] = lines[1].find("span", class_="Data03").text.strip()
            row[f"{idx+1}走前_着順"] = lines[2].find("span", class_="Data04").text.strip()
            row[f"{idx+1}走前_頭数"] = lines[2].find("span", class_="Data05").text.strip()
            row[f"{idx+1}走前_枠番"] = lines[2].find("span", class_="Data06").text.strip()
            row[f"{idx+1}走前_人気"] = lines[2].find("span", class_="Data07").text.strip()
            
            ##
            row[f"{idx+1}走前_馬場"] = lines[3].find("span", class_="Data09").text.strip()[0]
            row[f"{idx+1}走前_距離"] = lines[3].find("span", class_="Data09").text.strip()[1:]
            row[f"{idx+1}走前_コース"] = lines[3].find("span", class_="Data10").text.strip()
            row[f"{idx+1}走前_馬場状態"] = lines[3].find("span", class_="Data11").text.strip()
            row[f"{idx+1}走前_タイム"] = lines[3].find("span", class_="Data12").text.strip()
            ##
            row[f"{idx+1}走前_騎手"] = lines[4].find("span", class_="Data14").text.strip()
            row[f"{idx+1}走前_斤量"] = lines[4].find("span", class_="Data15").text.strip()
            row[f"{idx+1}走前_馬体重"] = lines[4].find("span", class_="Data16").text.strip().replace("kg", "")
            row[f"{idx+1}走前_増減"] = lines[4].find("span", class_="Data17").text.strip().replace("(", "").replace(")", "")

            row[f"{idx+1}走前_前半"] = lines[5].find("span", class_="Data19").text.strip()
            corners = lines[5].find("span", class_="Data20")
            row[f"{idx+1}走前_コーナー通過"] = " ".join([c.text for c in corners.find_all("span", class_="Corner")])
            row[f"{idx+1}走前_後半"] = lines[5].find("span", class_="Data21").text.strip()
            
            row[f"{idx+1}走前_ペース"] = lines[6].find("span", class_="Data13").text.strip()
            horse_link = lines[6].find("span", class_="Data22").find("a")
            row[f"{idx+1}走前_馬名"] = horse_link.text.strip() if horse_link else ""
            row[f"{idx+1}走前_着差"] = lines[6].find("span", class_="Data23").text.strip().replace("(", "").replace(")", "")
            row[f"{idx+1}走前_脚質"] = lines[6].find("span", class_="Data24").text.strip()

    except Exception as e:
        print(f"idx:{idx}, エラー:{e}")

data_rows.append(row)

In [106]:
pd.DataFrame(data_rows)

,1走前_日付_場所_レース,1走前_クラス,1走前_レース名,1走前_ハンデ,1走前_着順,1走前_頭数,1走前_枠番,1走前_人気,2走前_日付_場所_レース,2走前_クラス,...,4走前_枠番,4走前_人気,5走前_日付_場所_レース,5走前_クラス,5走前_レース名,5走前_ハンデ,5走前_着順,5走前_頭数,5走前_枠番,5走前_人気
0,06/29 小倉 11R,3勝,佐世保Ｓ,ﾊﾝﾃﾞ,1,13,3,2,None,None,...,7,5,12/08 中京 11R,3勝,知立Ｓ,定量,3,13,7,3
